In [4]:
import OSGridConverter #To convert from =SGB36 to WGS84
import pandas as pd #To use pandas for elegant data handling
import numpy as np
import math
import plotly.express as px
import pickle
from pathlib import Path
import spacy

First, we need to run the three classes below (`SpatialIndex`, `Postings` and `Gazetteer`). These are required to do all the work.

We read in the data first to create our corpus.

In [5]:
data_folder = Path('./data')
fn1 = data_folder / 'geograph_mini_corpus.csv'

geograph = pd.read_csv(fn1, encoding='latin-1')
        
#sample = geograph.sample(n = 1000) # For testing
sample = geograph

print(f'The initial corpus contains {len(sample)} documents.')

The initial corpus contains 138639 documents.


Now we create the three indexes we need:

- The spatial index stores documents in square cells and allows us to perform a range query
- The postings index indexes terms and the documents in which they are found
- The gazetteer indexes toponyms and allows us to look them up

The spatial index takes a resolution as a parameter
The postings file is restricted to terms from this [paper](https://firstmonday.org/ojs/index.php/fm/article/view/3710/3035) if the argument is true. If false then all terms (including stop words) are indexed.

Building these indexes takes time, but only needs to be done once. To make the code run faster in Binder, I use pickles (basically files that store the binary representation of the indexes, after creation). If we change `changes` to be `True` then the indexes are recalculated, but this is **slow**.

In [6]:
# I use pickles here to load objects that haven't changed. That is much quicker than rebuilding
# the indexes from scratch, especially the postings one (where I do NLP on all of our documents)
changes = False
data_folder = Path('./data')
fn1 = data_folder / 'si.obj'
fn2 = data_folder / 'postings.obj'
fn3 = data_folder / 'gaz.obj'
nlp = spacy.load('en_core_web_md')

if changes:
    resolution = 10000
    limitedTerms = True
    si = SpatialIndex(resolution, sample)
    postings = Postings(limitedTerms, sample, nlp)
    gaz = Gazetteer()
    f1 = open(fn1, 'wb')
    pickle.dump(si,f1)
    f1.close()
    f2 = open(fn2, 'wb')
    pickle.dump(postings,f2)
    f2.close()
    f3 = open(fn3, 'wb')
    pickle.dump(gaz,f3)
    f3.close()
else:
    f1 = open(fn1, 'rb')
    si = pickle.load(f1)
    f1.close()
    f2 = open(fn2, 'rb')
    postings = pickle.load(f2)
    f2.close()
    f3 = open(fn3, 'rb')
    gaz = pickle.load(f3)
    f3.close()

Demonstrates a look up in the gazetteer. For some toponym types point and bounding box are returned. Bounding box could be used to exclude documents for near queries. Location is returned as a tuple, and the first two elements are used to query the spatial index.

In [65]:
name = 'London'
location = gaz.getLocation(name)
print(location)

['London' 181500 531500 'City of London' 'C']
(531500, 181500, 529500, 179500, 533500, 183500)


Find all documents within a bounding box "centred" around the location returned from the gazetteer. Documents are returned with their distances and ranked according to distance from toponym. 

In [66]:
maxDist = 100000
docsSpatial = si.rangeQuery(maxDist, (location[0],location[1]))
print(f'Found {len(docsSpatial)} documents in a {maxDist}m x {maxDist}m around {name}.')

Found 8084 documents in a 100000m x 100000m around London.


Do a query and return documents ranked by tf-idf scores in the corpus.

In [67]:
query = 'hill big rough'
docsThematic = postings.tfIdf(query, nlp)
maxThematic = next(iter(docsThematic.items()))[1] # Used to normalise scores
print(f'Found {len(docsThematic)} documents using the query: {query}')

Found 20489 documents using the query: hill big rough


Find the unique documents from both sets. Typically there will be some duplicates, so the sum of the two previous numbers is the maximum value we can find here.

In [68]:
#Create a set of unique keys for all the documents retrieved from the thematic and spatial indexes
candidates = set(list(docsSpatial.keys()) + list(docsThematic.keys()))

print(f'Found {len(candidates)} unique documents merging the thematic and spatial indexes')

Found 27965 unique documents merging the thematic and spatial indexes


Rank documents according to spatial (distance) and thematic (tf-idf) scores. `wspatial` determines the **weight** of each component. A **value of 1.0** means that the ranking is purely **spatial**, **0.0** considers only **tf-idf scores**.

In [69]:
wspatial = 0.5 # Weight of the spatial score -> weight of 1 means tfidf is ignored

scores = dict()
for doc in candidates:
    st = 0
    ss = 0
    if doc in docsThematic:
        st = docsThematic[doc]/ maxThematic
        #print(f'thematic {st}')
    if doc in docsSpatial:
        ss = 1- docsSpatial[doc]/ maxDist
        #print(f'spatial {docsSpatial[doc]} {ss}')
    score = ((1-wspatial) * st) + (wspatial * ss)
    scores[doc] = score
ranked = dict(sorted(scores.items(), key = lambda x: x[1], reverse=True))

This section of code outputs the results in order. It retrieves the document text from the original data frame to display, not that we didn't use this up to now.

In [70]:
n = 100
i=0
for key in ranked:
    score = ranked[key]
    row = sample.loc[sample['id'] == key]
    title = row.iloc[0]['title']
    text = row.iloc[0]['text']
    print(f'{score:.2f}: {title} - {text}\n')
    if i == n:
        break
    i = i+1

0.67: Hammersmith Town Hall in daylight - Hammersmith Town Hall this time taken on the left of the tree. The bigger building to the left and back of the 1930's version is the soon to be demolished modern main entrance.

0.65: Fallen tree by Western Avenue - This tree fell down in big storms the day before. The railway beyond is the Central Line.

0.62: Ever hopeful hungry water birds - At Hainault Forest Country Park this collection of water fowl were ever hopeful. Luckily ''Mums and Dads'' and ''Grandparents'' with their ''offspring'' were present with bags of unused or stale bread to replenish food stocks. Why is it small hands always seem to have big pieces to toss towards the eager beaks rather the dainty bits held by their elder stewards?

0.60: Pashley Manor oak tree opposite car park - My big regret was not measuring the girth of this tree. If the Sussex post and rail fencing behind it is set at 6 foot centres then the tree could be 5 or 6 feet in diameter. The former would equa

In [71]:
import geopandas as gpd
from shapely.geometry import Point
i=0


df = pd.DataFrame()

for key in ranked:
    row = sample.loc[sample['id'] == key]
    t = pd.DataFrame(
        {
            'lat': row.iloc[0]['lat'],
            'lon': [row.iloc[0]['lon']],
            'score': [ranked[key]],
            'title': [row.iloc[0]['title']]
        }
    )

    df = pd.concat([df, t])

    if i == n:
        break
    i = i+1

df

,lat,lon,score,title
0,51.491779,-0.233868,0.667730,Hammersmith Town Hall in daylight
0,51.530867,-0.298820,0.646966,Fallen tree by Western Avenue
0,51.613480,0.127329,0.617152,Ever hopeful hungry water birds
0,51.036166,0.433847,0.601278,Pashley Manor oak tree opposite car park
0,51.472709,-0.008968,0.601217,A2 Blackheath Hill
...,...,...,...,...
0,51.513209,-0.108808,0.497640,City of London: Kings Bench Walk
0,51.513715,-0.101149,0.497422,Ludgate Hill
0,51.513713,-0.101005,0.497387,Ludgate Hill
0,51.513268,-0.101311,0.497266,Ludgate Hill looking towards Creed Lane


In [72]:
fig = px.scatter_mapbox(df, lat="lat", 
                     lon="lon",
                     color="score", # which column to use to set the color of markers
                     hover_name="title", # column added to hover information
                     zoom=6,
                     height=600,
                    )
fig.update_layout(mapbox_style="open-street-map")
fig.show()


In [1]:
class SpatialIndex:
    
    def __init__(self, resolution, sample):
        
        sample.dropna() # Get rid of problematic rows with nas
        
        for i in sample.index:
            try:
                g = OSGridConverter.latlong2grid (sample.at[i, 'lat'], sample.at[i, 'lon'], tag = 'WGS84')
                sample.at[i, 'x'] = g.E
                sample.at[i, 'y'] = g.N
            except ValueError:
                #print("Problem with a document", sample.at[i,'id'])
                sample = sample.drop(i)

        # Now we can set up the parameters for our index        
        self.resolution = resolution

        self.minx = sample['x'].min()
        self.maxx = sample['x'].max()
        self.miny = sample['y'].min()
        self.maxy = sample['y'].max()

        w = self.maxx - self.minx
        h = self.maxy - self.miny

        nc = int(w/self.resolution) + 1
        nr = int(h/self.resolution) + 1

        #print(maxx, minx, maxy, miny)
        #print(nr, nc)

        #Build the spatial index now
        self.spatialIndex = pd.DataFrame(index=range(nc),columns=range(nr))

        #Now we populate the index with document ids
        for index, row in sample.iterrows():
            i = int((row['x'] - self.minx)/self.resolution)
            j = int((row['y'] - self.miny)/self.resolution)
            id = row['id']
    
            #print(row['id'])
            #print(row['x'],row['y'],i,j)
            if pd.isnull(self.spatialIndex.at[i,j]):
                self.spatialIndex.at[i,j] = {id:(row['x'],row['y'])}
            else:
                names = self.spatialIndex.at[i,j]
                names.update({id:(row['x'],row['y'])})
                self.spatialIndex.at[i,j] = names

        
    def rangeQuery(self, dist, point):
        x1 = point[0] - dist/2
        x2 = point[0] + dist/2
        y1 = point[1] - dist/2
        y2 = point[1] + dist/2
    
        i1 = int((x1 - self.minx)/self.resolution)
        j1 = int((y1 - self.miny)/self.resolution)
        i2 = int((x2 - self.minx)/self.resolution) + 1
        j2 = int((y2 - self.miny)/self.resolution) + 1

        # Retrieve only the relevant part of the index
        result = self.spatialIndex.iloc[i1:i2, j1:j2]
        # Turn the data frame into a 1d list
        tlist = result.values.flatten()
        # Remove all the nans
        filtered = filter(lambda i:not(type(i) is float), tlist)
        
        #Rank by distance
        ranked = {}
        for item in filtered:
            for key in item:
                d = si.dist(point, item[key])
                #print(key, item[key], dist)
                ranked[key] = d    
        ranked = dict(sorted(ranked.items(), key = lambda x: x[1], reverse=False))
                
        return ranked
    
    def dist(self, p1, p2):
        #print(p1[0], p1[1], p2[0], p2[1])
        dist = (((p1[0] - p2[0]) ** 2) + ((p1[1] - p2[1]) ** 2)) ** 0.5
        #print(dist)
        return dist

In [2]:
class Postings:
    
    def __init__(self, firstMondayTerms, sample, nlp):
        #Load a language model to do NLP
        self.ndocs = len(sample)
                
        # firstMonday works like an inverse stop list, and we only use words in these lists for our posting file
        if firstMondayTerms:
            #We do this so that the serialisation of the Pickle works propery
            data_folder = Path('./data')
            fn1 = data_folder / 'elements.txt'
            fn2 = data_folder / 'qualities.txt'
            fn3 = data_folder / 'activities.txt'
            
            elements = set(pd.read_csv(fn1, header=None)[0])
            qualities = set(pd.read_csv(fn2, header=None)[0])
            activities = set(pd.read_csv(fn3, header=None)[0])

            terms = elements.union(qualities).union(activities)
            lemmas = ' '.join(str(e) for e in terms)

            doc = nlp(lemmas)
            terms = set()
            for token in doc:
                terms.add(token.lemma_)
                
            # Now we process our corpus and create a postings file
            docs = nlp.pipe(sample.text,n_process=2, batch_size=100)

            self.postings = dict()

            for (idxRow, s1), (_, s2) in zip(sample.iterrows(), enumerate(docs)):
                id = s1.id
                for token in s2:
                    lemma = token.lemma_
                    if lemma in terms:

                        if lemma in self.postings:
                            tf = self.postings[lemma]
                            if id in tf:
                                tf[id] = tf[id] + 1
                            else:
                                tf[id] = 1
                        else:
                            tf = {id: 1}
                        self.postings[lemma] = tf
                        
    def tfIdf(self, query,nlp):
        results = {}
        qdoc = nlp(query)
        for token in qdoc:
            qt = token.lemma_
            if qt in self.postings:
                dc = len(self.postings[qt])
                idf = math.log10(self.ndocs/(dc + 1))
                for doc in self.postings[qt]:
                    tf = self.postings[qt][doc]
                    tfidf = tf * idf
                    if doc in results:
                        score = results[doc]
                        results[doc] = tfidf + score
                    else:
                        results[doc] = tfidf
        results = dict(sorted(results.items(), key = lambda x: x[1], reverse=True))
        
        return results

In [3]:
# Feature codes in gazetteer are as follows:
# A Antiquity (non-Roman)
# F Forest or wood
# FM Farm
# H Hill or mountain
# R Antiquity (Roman)
# C City
# T Town
# O Other
# W Water feature
# X All other features

class Gazetteer:
    
    def __init__(self):
        self.gaz = dict()
        self.offset = {'C': 2000, 'T':500, 'H':250, 'F':500}
        # Read in gazetteer data
        data_folder = Path('./data')
        fn1 = data_folder / '50kgaz2012.txt'
        os_50k = pd.read_csv(fn1,sep=':', encoding='utf8', header=None)
        os_trimmed = os_50k.drop([0,1,3,4,5,6,7,10,11,12,15,16,17,18,19], axis = 1)
        os_trimmed.columns = ['name','y','x','county','type']
        for index, row in os_trimmed.iterrows():
            name = row['name']
            entry = os_trimmed.iloc[index].values 
            # Store gazetteer in a dictionary of unique names
            if name in self.gaz:
                entries = self.gaz[name]
                entries.append(entry)
                self.gaz[name] = entries
            else:
                self.gaz[name] = [entry]
            
    def getLocation(self, name):
        if (name in self.gaz) == False:
            return('Name not found in gazetteer')

        if len(self.gaz[name]) > 1:
            # We let the user disambiguate
            i = 0
            print("This place name is ambiguous - choose an entry")
            for entry in self.gaz[name]:
                print(f'{i}: {name}, {entry[3]}')
                i = i + 1
            index = int(input("Choose a value:"))
            entry = self.gaz[name][index]
        else:
            entry = self.gaz[name][0]
            
        print(entry)
        x = entry[2]
        y = entry[1]
            
        if entry[4] in self.offset:
            diff = self.offset[entry[4]]
            return (x,y,x-diff, y-diff, x + diff, y + diff)
        else:
            return(x,y)
                                    
    def gazDump(self):
        for name in self.gaz:
            print(name)
            print(self.gaz[name])
            

In [6]:
#output all dependencies so that we can reproduce the notebook (we only need this to set things up for Binder)
#%load_ext watermark
#%watermark --iversions